# Data downloader


In [1]:
!pip install tokenizers -q
!pip install transformers -q
!pip install datasets -q

     |████████████████████████████████| 7.6 MB 13.4 MB/s 
     |████████████████████████████████| 5.8 MB 15.2 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 451 kB 13.5 MB/s 
     |████████████████████████████████| 132 kB 80.5 MB/s 
     |████████████████████████████████| 212 kB 66.4 MB/s 
     |████████████████████████████████| 127 kB 73.3 MB/s 


In [2]:
from datasets import load_dataset

dataset = load_dataset("trec")

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset["train"][0]

{'text': 'How did serfdom develop in and then leave Russia ?',
 'coarse_label': 2,
 'fine_label': 26}

In [4]:
num_labels = len(dataset["train"].features["coarse_label"].names)

In [5]:
num_labels

6

# Preprocessing

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased", use_fast = True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
tokenizer(dataset["train"][0]["text"])

{'input_ids': [101, 1731, 1225, 14516, 11931, 9277, 3689, 1107, 1105, 1173, 1817, 2733, 136, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation = True)

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched = True, remove_columns = ["fine_label"])

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Rename the label column to fit the concept

In [11]:
tokenized_dataset = tokenized_dataset.rename_column("coarse_label", "labels")

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

# Finetuning BERT for text classification

In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels = num_labels)

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bia

In [15]:
batch_size = 16
learning_rate = 2e-5
weight_decay = .01
num_train_epochs = 5

In [16]:
args = TrainingArguments(
    "/saved",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_train_epochs,
    weight_decay = weight_decay,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    # push_to_hub = True
)

In [17]:
from datasets import load_metric
import numpy as np

In [18]:
metric = load_metric("glue", "mnli")

<ipython-input-18-1c23e2ff899a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mnli")


In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis = 1)

    return metric.compute(predictions = predictions, references = labels)

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.267638,0.926000
2,0.449100,0.149218,0.960000
3,0.100400,0.141545,0.966000
4,0.100400,0.148933,0.968000
5,0.035100,0.138593,0.972000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to /saved/checkpoint-1705
Configuration saved in /saved/checkpoint-1705/config.json
Model weights saved in /saved/checkpoint-1705/pytorch_model.bin
tokenizer config file saved in /saved/checkpoint-1705/tokenizer_config.json
Special tokens file saved in /saved/checkpoint-1705/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /saved/checkpoint-1705 (score: 0.972).


TrainOutput(global_step=1705, training_loss=0.1745173208175167, metrics={'train_runtime': 121.0621, 'train_samples_per_second': 225.174, 'train_steps_per_second': 14.084, 'total_flos': 169213299099120.0, 'train_loss': 0.1745173208175167, 'epoch': 5.0})

# Evaluation

In [22]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16


{'eval_loss': 0.13859347999095917,
 'eval_accuracy': 0.972,
 'eval_runtime': 0.4701,
 'eval_samples_per_second': 1063.558,
 'eval_steps_per_second': 68.068,
 'epoch': 5.0}

In [23]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis = 1)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 500
  Batch size = 16


In [24]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [25]:
print(classification_report(predictions, labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.89      0.97      0.93        87
           2       1.00      0.97      0.98       143
           3       0.98      0.98      0.98        65
           4       0.98      0.96      0.97        82
           5       0.99      0.98      0.99       114

    accuracy                           0.97       500
   macro avg       0.97      0.98      0.98       500
weighted avg       0.97      0.97      0.97       500



In [26]:
print(f"Accuracy = {accuracy_score(predictions, labels):.6f}, \
        Precision = {precision_score(predictions, labels, average = 'macro'):.6f}, \
        Recall = {recall_score(predictions, labels, average = 'macro'):.6f}, \
        F1 = {f1_score(predictions, labels, average = 'macro'):.6f}")

Accuracy = 0.972000,         Precision = 0.974115,         Recall = 0.976840,         F1 = 0.975185


In [27]:
print(confusion_matrix(predictions, labels))

[[  9   0   0   0   0   0]
 [  0  84   0   1   2   0]
 [  0   4 138   0   0   1]
 [  0   1   0  64   0   0]
 [  0   3   0   0  79   0]
 [  0   2   0   0   0 112]]


In [28]:
def print_clarification(idx):
    for sent, pred, label in zip(tokenized_dataset["test"][0:idx]["text"], predictions[0:idx], labels[0:idx]):
        print(sent)
        print(f"Predicted: {pred} - Label: {label}")

In [29]:
print_clarification(10)

How far is it from Denver to Aspen ?
Predicted: 5 - Label: 5
What county is Modesto , California in ?
Predicted: 4 - Label: 4
Who was Galileo ?
Predicted: 3 - Label: 3
What is an atom ?
Predicted: 2 - Label: 2
When did Hawaii become a state ?
Predicted: 5 - Label: 5
How tall is the Sears Building ?
Predicted: 5 - Label: 5
George Bush purchased a small interest in which baseball team ?
Predicted: 3 - Label: 3
What is Australia 's national flower ?
Predicted: 1 - Label: 1
Why does the moon turn orange ?
Predicted: 2 - Label: 2
What is autism ?
Predicted: 2 - Label: 2


In [33]:
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        print(tokenized_dataset["test"][i]["text"])
        print(f"Predicted: {predictions[i]} - Label: {labels[i]}")

What is the speed hummingbirds fly ?
Predicted: 2 - Label: 5
What is the longest major league baseball-winning streak ?
Predicted: 5 - Label: 1
What is the major fault line near Kentucky ?
Predicted: 4 - Label: 1
In the late 1700 's British convicts were used to populate which colony ?
Predicted: 1 - Label: 4
What is the birthstone for June ?
Predicted: 2 - Label: 1
What is the sales tax in Minnesota ?
Predicted: 5 - Label: 1
What New York City structure is also known as the Twin Towers ?
Predicted: 1 - Label: 4
What are the spots on dominoes called ?
Predicted: 4 - Label: 1
What is the electrical output in Madrid , Spain ?
Predicted: 2 - Label: 1
What are the two houses of the Legislative branch ?
Predicted: 3 - Label: 1
What is the criterion for being legally blind ?
Predicted: 2 - Label: 1
What is foot and mouth disease ?
Predicted: 2 - Label: 1
What monastery was raided by Vikings in the late eighth century ?
Predicted: 4 - Label: 1
What did Jesse Jackson organize ?
Predicted: 1 - 